In [40]:
import re
import pandas as pd
import numpy as np

import fasttext

In [101]:
def allmax(a: np.array, limit: int = 0.95) -> list:
    if len(a) == 0:
        return []
    all_ = []
    for i in range(len(a)):
        if a[i] >= limit:
            all_.append(i)
    return all_


def allmin(a: np.array, limit: int = 0.95) -> list:
    if len(a) == 0:
        return []
    all_ = []
    for i in range(len(a)):
        if a[i] <= limit:
            all_.append(i)
    return all_
# https://stackoverflow.com/questions/17568612/how-to-make-numpy-argmax-return-all-occurrences-of-the-maximum

# Data

Это некие промежуточные файлы, но там как раз финальный результат разметки. Synonyms и Excludes надо объединить.

Topic - тема, надо брать из поля Synonyms то, где в Result стоит 1  (0 - это блокирующие слова и фразы, типа мы считаем что слово "агрессивный" определяет "Агрессивный аромат", но если "агрессивная реклама" - то не хотим выделять эту тему).

Parfjum_full_list_to_markup - общий список, в том числе то что не имеет темы

In [181]:
excludes = pd.read_csv('data/input/Parfjum_CL_Excludes.csv')
excludes.head()

,Unnamed: 0,Unnamed: 0.1,Topic,Stemma,Excludes,Result
0,0,0,Агрессивный,агрессив,агрессивная красотка,0
1,1,1,Аккорд,аккорд,абстрактный цветочный аккорд,1
2,2,2,Аккорд,аккорд,аккорд абсолюта,1
3,3,3,Аккорд,аккорд,аккорд амбры,1
4,4,4,Аккорд,аккорд,аккорд ананаса,1


In [182]:
synonyms = pd.read_csv('data/input/Parfjum_CL_Synonyms.csv')
synonyms.head()

,Unnamed: 0,Unnamed: 0.1,Topic,Stemma,Synonyms,Result
0,0,0,Агрессивный,агресивн,агресивны,1
1,1,1,Агрессивный,агрессив,агрессивен,1
2,2,2,Агрессивный,агрессив,агрессивна,1
3,3,3,Агрессивный,агрессив,агрессивная,1
4,4,4,Агрессивный,агрессив,агрессивная нота,1


In [183]:
full = pd.read_csv('data/input/Parfjum_full_list_to_markup.csv')
full.head()

,Unnamed: 0,Unnamed: 0.1,item,frequency,words_ordered,stemmed_text,count_words
0,0,0,NaN,2084504,NaN,NaN,0
1,1,231304,-а,3,-а,-а,1
2,2,259046,-а-,3,-а-,-а-,1
3,3,224002,-а-л-и-м-а,3,-а-л-и-м-а,-а-л-и-м-,1
4,4,336287,нота -бергамот,5,-бергамот нота,-бергамот нот,2


### Обработка данных для разметки

In [109]:
class ClearingPhrases:
    regex = re.compile("[А-ЯЁа-яё]+")

    def __init__(self, texts):
        self.texts = texts

    def words_only(self, text):
        try:
            return self.regex.findall(text.lower())
        except AttributeError:
            return []

    @property
    def get_best_texts(self):
        texts = []
        for text in self.texts:
            split_w = self.words_only(text)
            exit = 0
            if len(split_w) >= 2:
                for word in split_w:
                    if len(word) <= 6:
                        exit = 1
                        break
                if exit:
                    continue
                texts.append(' '.join(split_w))
        return list(set(texts))

In [181]:
phrases = ClearingPhrases(full.item.values).get_best_texts
phrases[:5]

['горьковато сладкое',
 'арабский парфюмом',
 'белоцветочная композиция',
 'цитрусовая корочка',
 'дешевый парфюмы']

### Данные для обучения классификатора

**Создание обучающего набора**

In [184]:
m = synonyms.merge(full, how="inner", left_on='Synonyms', right_on='item').drop_duplicates()
m = m.loc[m['Result'] == 1, ['item', 'Topic', 'frequency']]
m.head()

,item,Topic,frequency
0,агресивны,Агрессивный,7
1,агрессивен,Агрессивный,92
2,агрессивна,Агрессивный,28
3,агрессивная,Агрессивный,184
4,агрессивная нота,Агрессивный,8


In [188]:
m.sort_values('frequency', ascending=False).to_csv('data/processed/perfumery_train.csv', index=False)

**Набор для решения проблемы холодного старта**

In [131]:
df = pd.read_csv('data/input/Parfjum_classifier.csv')
df.head()

,Категория,Тема,Подтема,Unnamed: 3
0,Описание звучания/нот аромата,Амбровые,Амбра/Амбровый,NaN
1,NaN,Древесные,Аромат смолы,NaN
2,NaN,Зеленые,Зелени,NaN
3,NaN,NaN,Аромат листьев,NaN
4,NaN,NaN,Хвойный аромат,NaN


In [155]:
df.loc[df['Тема'].isna() & df['Категория'].notna(), 'Тема'] = df.loc[df['Тема'].isna() & df['Категория'].notna(), 'Категория']
df['Тема'].fillna(method='pad', inplace=True)
df.loc[df['Подтема'].isna(), 'Подтема'] = df.loc[df['Подтема'].isna(), 'Тема'] 

In [156]:
pd.DataFrame({'phrase': parfjum['Подтема'].unique(), 'subtopic': parfjum['Подтема'].unique()})

,phrase,subtopic
0,Амбра/Амбровый,Амбра/Амбровый
1,Аромат смолы,Аромат смолы
2,Зелени,Зелени
3,Аромат листьев,Аромат листьев
4,Хвойный аромат,Хвойный аромат
...,...,...
152,Условия хранения,Условия хранения
153,Срок годности,Срок годности
154,Место покупки,Место покупки
155,Название аромата,Название аромата


In [157]:
subtopics = parfjum['Подтема'].unique()
len(subtopics)

157

# Классификатор

In [115]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle
import fasttext
from sklearn.neighbors import KNeighborsClassifier

In [9]:
model = fasttext.load_model("models/adaptation/perfumery.bin")

In [5]:
with open(os.path.join(os.getcwd(), "models", 'classifier.pkl'), 'rb') as model:
    classifier = pickle.load(model)

### Подготовка данных

In [189]:
data = pd.read_csv('data/processed/perfumery_train.csv')
data.head()

,item,Topic,frequency
0,ноты,Нота,126461
1,стойкий,Стойкость,90695
2,флакон,Флакон,88474
3,духи,Продукт,69311
4,стойкость,Стойкость,58420


In [190]:
data.shape

(35262, 3)

In [8]:
X = data.sort_values('frequency', ascending=False)['item']
y = data.sort_values('frequency', ascending=False)['Topic']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)

In [12]:
X_train = np.array([model.get_sentence_vector(i) for i in X_train])
X_test = np.array([model.get_sentence_vector(i) for i in X_test])

In [117]:
X = np.array([model.get_sentence_vector(i) for i in X])

In [118]:
classifier = KNeighborsClassifier(n_neighbors=5, weights='distance', n_jobs=-1, metric='cosine')
classifier.fit(X, y)

KNeighborsClassifier(metric='cosine', n_jobs=-1, weights='distance')

In [120]:
len(classifier.classes_)

175

In [13]:
y_pred = classifier.predict(X_test)

In [14]:
from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

                          precision    recall  f1-score   support

             Агрессивный       0.00      0.00      0.00         6
                  Аккорд       0.00      0.00      0.00        36
    Активный образ жизни       0.00      0.00      0.00         1
                  Акцент       0.00      0.00      0.00         8
         Алкоголя/спирта       0.00      0.00      0.00        47
   Аллергическая реакция       0.33      0.33      0.33         3
               Альдегида       0.00      0.00      0.00         9
          Амбра/Амбровый       0.00      0.00      0.00         0
          Амбра/амбровый       0.00      0.00      0.00        36
              Аппликатор       0.00      0.00      0.00         2
             Аромат дыма       0.04      0.33      0.07        18
          Аромат листьев       0.01      0.50      0.02        12
            Аромат смолы       0.01      0.53      0.02        19
           Аромат табака       0.00      0.00      0.00        31
         

C:\Users\bimas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bimas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bimas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\bimas\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [46]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# Assign colum names to the dataset
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

# Read dataset to pandas dataframe
dataset = pd.read_csv(url, names=names)
dataset.head()

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [47]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 4].values

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [57]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [117]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=10, weights='distance', n_jobs=-1)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_jobs=-1, n_neighbors=50, weights='distance')

In [118]:
y_pred = classifier.predict(X_test)

In [119]:
classifier.predict_proba([X_test[0]])

array([[0.        , 0.70966769, 0.29033231]])

In [120]:
classifier.predict([X_test[0]])

array(['Iris-versicolor'], dtype=object)

In [121]:
pd.Series(y).unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [122]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 7  0  0]
 [ 0 12  1]
 [ 0  0 10]]
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00         7
Iris-versicolor       1.00      0.92      0.96        13
 Iris-virginica       0.91      1.00      0.95        10

       accuracy                           0.97        30
      macro avg       0.97      0.97      0.97        30
   weighted avg       0.97      0.97      0.97        30

